In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [2]:
data = pd.read_excel('C:\\Users\\user\\Downloads\\Report.xlsx')

In [3]:
data

,기간,구분,구분.1,전체,남자,여자
0,2009,서울시,서울시,24.3,45.5,4
1,2009,생애주기별,19~29세,24.9,43.8,6
2,2009,생애주기별,30~44세,29.9,55.1,3.8
3,2009,생애주기별,45~64세,22.6,43.2,3.3
4,2009,생애주기별,65세 이상,11.5,22.8,2.9
...,...,...,...,...,...,...
424,2019,관악구,관악구,18.0,30.3,5.6
425,2019,서초구,서초구,12.6,25.1,1.5
426,2019,강남구,강남구,14.7,26.6,4.2
427,2019,송파구,송파구,14.9,28.8,F


In [11]:
data['여자'].unique()

array([4, 6, 3.8, 3.3, 2.9, 4.9, 4.4, 1.9, nan, 5.8, 2.7, 5.3, 5.4, 3.5,
       5.5, 3.1, 4.6, 2.3, 3.7, 3.2, 2.8, 5.7, 4.5, 5.1, 3.6, 4.8, 4.7,
       4.1, 7.3, 5, 5.2, 2.5, 2.4, 2.6, 3, 4.2, 3.9, 2, 3.4, 4.3, 7.4,
       2.1, 2.2, 1.5, 1.7, 5.6, 1.8, 7, 7.8, 6.6, 1.6, 1.4, 6.1, 6.4, 1,
       6.2], dtype=object)

In [10]:
data.loc[(data['여자'] == 'F') | (data['여자'] == '…') | (data['여자'] == '-'), '여자'] = np.nan

In [28]:
data['여자'] = data['여자'].astype(float)

In [29]:
data

,기간,구분,구분.1,전체,남자,여자
0,2009,서울시,서울시,24.3,45.5,4.0
1,2009,생애주기별,19~29세,24.9,43.8,6.0
2,2009,생애주기별,30~44세,29.9,55.1,3.8
3,2009,생애주기별,45~64세,22.6,43.2,3.3
4,2009,생애주기별,65세 이상,11.5,22.8,2.9
...,...,...,...,...,...,...
424,2019,관악구,관악구,18.0,30.3,5.6
425,2019,서초구,서초구,12.6,25.1,1.5
426,2019,강남구,강남구,14.7,26.6,4.2
427,2019,송파구,송파구,14.9,28.8,NaN


In [30]:
# 구별 데이터만 추출

data_gu = data.loc[data['구분'].str.endswith("구")].reset_index(drop = True)
data_gu

,기간,구분,구분.1,전체,남자,여자
0,2009,종로구,종로구,23.0,41.2,5.3
1,2009,중구,중구,27.0,48.7,5.3
2,2009,용산구,용산구,25.1,46.2,5.4
3,2009,성동구,성동구,25.3,47.3,3.5
4,2009,광진구,광진구,24.3,44.0,5.4
...,...,...,...,...,...,...
270,2019,관악구,관악구,18.0,30.3,5.6
271,2019,서초구,서초구,12.6,25.1,1.5
272,2019,강남구,강남구,14.7,26.6,4.2
273,2019,송파구,송파구,14.9,28.8,NaN


In [47]:
# 년도별 나이대별 데이터

data_year_age = data.loc[data['구분'] == '생애주기별'].reset_index(drop = True)
data_year_age

,기간,구분,구분.1,전체,남자,여자
0,2009,생애주기별,19~29세,24.9,43.8,6.0
1,2009,생애주기별,30~44세,29.9,55.1,3.8
2,2009,생애주기별,45~64세,22.6,43.2,3.3
3,2009,생애주기별,65세 이상,11.5,22.8,2.9
4,2010,생애주기별,19~29세,23.8,43.2,4.4
5,2010,생애주기별,30~44세,28.8,53.4,3.5
6,2010,생애주기별,45~64세,21.4,40.6,3.6
7,2010,생애주기별,65세 이상,11.4,21.9,2.9
8,2011,생애주기별,19~29세,22.5,40.0,4.9
9,2011,생애주기별,30~44세,28.6,52.3,4.0


In [49]:
data_year_age.drop(columns=['구분'], inplace = True)
data_year_age.rename(columns = {'구분.1':'나이대'}, inplace = True)

In [50]:
data_year_age.head()

,기간,나이대,전체,남자,여자
0,2009,19~29세,24.9,43.8,6.0
1,2009,30~44세,29.9,55.1,3.8
2,2009,45~64세,22.6,43.2,3.3
3,2009,65세 이상,11.5,22.8,2.9
4,2010,19~29세,23.8,43.2,4.4


In [52]:
data_year_age.to_csv("D:\\SNUlab\\0. data\\서울시 흡연율\\smoking_year_age.csv", encoding = 'euc-kr', index = False)

In [31]:
data_gu['구분'].unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [33]:
data_year = data_gu.groupby('기간').mean()
data_year

,전체,남자,여자
기간,,,
2009,24.508,45.636,4.254545
2010,23.404,43.752,3.920833
2011,22.548,42.224,3.633333
2012,22.532,41.944,3.824000
2013,21.856,40.764,3.936364
2014,20.916,39.004,3.726087
2015,19.712,36.692,3.695238
2016,19.836,36.972,3.530435
2017,18.860,35.176,3.755000


In [36]:
data_year.to_csv("D:\\SNUlab\\0. data\\서울시 흡연율\\smoking_year_sex.csv", encoding = 'euc-kr')

In [43]:
data_year_gu = data_gu.groupby(['기간', '구분.1']).mean()
data_year_gu

전체    남자   여자
기간   구분.1                 
2009 강남구   19.4  36.4  4.5
     강동구   24.3  43.9  5.1
     강북구   26.7  48.6  5.5
     강서구   24.6  46.6  3.7
     관악구   27.7  49.2  5.7
...         ...   ...  ...
2019 용산구   15.1  28.2  3.1
     은평구   15.5  27.0  5.1
     종로구   18.5  33.5  4.4
     중구    17.7  31.8  4.1
     중랑구   20.1  36.7  3.9

[275 rows x 3 columns]

In [44]:
data_year_gu.to_csv("D:\\SNUlab\\0. data\\서울시 흡연율\\smoking_year_gu_sex.csv", encoding = 'euc-kr')